In [1]:
import numpy as np
import pandas as pd
from equiflow import TableZero

In [2]:
np.random.seed(42)
n = 100
data = {
    'age': np.random.randint(18, 80, size=n),
    'race': np.random.choice(['White', 'Black', 'Asian', 'Hispanic', None],
                             size=n),
    'sex': np.random.choice(['Male', 'Female'], size=n),
    'english': np.random.choice(['Fluent', 'Limited'], size=n),
}

for i in range(1, 11):
    data[f'var{i}'] = np.random.randn(n)

df = pd.DataFrame(data)

In [3]:
data_0 = df.copy()
data_1 = data_0.loc[data_0.sex == "Female"]
data_2 = data_1.loc[data_1.english == "Fluent"]

t = TableZero(dfs = [data_0, data_1, data_2],
              cols = ['race','sex', 'english'],
              format='N (%)'
              )

t1 = t.table_one()
t1

Cohort                        
                           0           1           2
Variable Value                                      
race     Black     13 (13.0)    8 (15.4)    3 (16.7)
         Hispanic  22 (22.0)    8 (15.4)    4 (22.2)
         White     25 (25.0)   11 (21.2)    5 (27.8)
         Asian     20 (20.0)   12 (23.1)    3 (16.7)
         Missing   20 (20.0)   13 (25.0)    3 (16.7)
sex      Female    52 (52.0)  52 (100.0)  18 (100.0)
         Male      48 (48.0)     0 (0.0)     0 (0.0)
         Missing     0 (0.0)     0 (0.0)     0 (0.0)
english  Limited   58 (58.0)   34 (65.4)     0 (0.0)
         Fluent    42 (42.0)   18 (34.6)  18 (100.0)
         Missing     0 (0.0)     0 (0.0)     0 (0.0)

In [4]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(t1)

Cohort                        
                           0           1           2
Variable Value                                      
race     Black     13 (13.0)    8 (15.4)    3 (16.7)
         Hispanic  22 (22.0)    8 (15.4)    4 (22.2)
         White     25 (25.0)   11 (21.2)    5 (27.8)
         Asian     20 (20.0)   12 (23.1)    3 (16.7)
         Missing   20 (20.0)   13 (25.0)    3 (16.7)
sex      Female    52 (52.0)  52 (100.0)  18 (100.0)
         Male      48 (48.0)     0 (0.0)     0 (0.0)
         Missing     0 (0.0)     0 (0.0)     0 (0.0)
english  Limited   58 (58.0)   34 (65.4)     0 (0.0)
         Fluent    42 (42.0)   18 (34.6)  18 (100.0)
         Missing     0 (0.0)     0 (0.0)     0 (0.0)

In [5]:
t1.loc[('race', 'Missing')]

Cohort  0    20 (20.0)
        1    13 (25.0)
        2     3 (16.7)
Name: (race, Missing), dtype: object